<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:60px; font-weight:bold;">CineClassify</div>
</div>
<br>
<p>In een wereld waar films in overvloed zijn, is er een groeiende behoefte om ze gemakkelijk en duidelijk te kunnen classificeren. Veel kijkers hebben geen goed overzicht over alle mengende genres en de vele nuances. Dit kan het zoeken van een leuke film voor op de vrijdagavond vanuit de Netflix catalogus of voor een trip naar de bios onnodig gecompliceerd maken.  

Daarom heeft het nieuwe filmplatform “CineClassify” als doel om films automatisch te laten classificeren in verschillende genres en een duidelijke gids te brengen naar de kijkers. Om dit doel te bereiken hebben ze een data-science team ingehuurd om een model te maken die genres kan gaan voorspellen. Aan de hand van verschillende gegevens, bijv. cast, regisseur, reviews, etc., moet het mogelijk worden gemaakt om filmliefhebbers over de wereld te helpen om eenvoudiger films te vinden die passen bij hun smaak. 

In dit notebook werken we aan onze opdracht van CineClassify. In dit notebook zal er een pipeline worden gebouwd om de data in te laden en er een dataframe van te maken. Deze dataframe kan worden gebruikt om de data duidelijk in te zien voor het datascience team.</p>
<br>
<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:40px; font-weight:bold;">Inhoudsopgave</div>
    <a name='begin'></a>
</div>

1. [Importeren van libaries](#start)
2. [IMDb Webscraping](#ws)
3. [Database: Movie Summaries](#db)
4. [API](#api)
5. [Preprocessing en Feature Engineering](#tr)
6. [Opzetten van de Pipeline](#pipe)
7. [Aantonen dat de Pipeline werkt](#toon)

<br>

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:40px; font-weight:bold;">Importeren van libaries</div>
    <a name='start'></a>
</div>

In [4]:
# Importeren standaard libaries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Importeren webscraping libaries
import requests
from bs4 import BeautifulSoup
import regex as re

# Importeren time-out libaries
from time import sleep
from random import randint

# Importeren Database libaries
import tarfile
import os
import urllib.request
from io import BytesIO
import sqlite3

# Importeren FE libaries
import re
from nltk.corpus import stopwords

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:40px; font-weight:bold;">IMDb Webscraping</div>
    <a name='ws'></a>
</div>

Om een deel van de data te krijgen is het nodig om data te verkrijgen van het internet. Dit wordt gedaan door middel van een techniek genaamd webscraping. Door middel van de BeautifulSoup library voor Python is het gemakkelijk gemaakt om deze stappen te ondernemen. Door het bekijken van de HTML code van de website kunnen de nodige elementen gevonden worden en kan de data opgehaald worden van de website. Voordat we beginnen met het coderen van de soup worden er eerst een paar nodige elementen aangemaakt.

Om deze data te verkrijgen is er gebruik gemaakt van een webscraper. Deze webscraper is helaas niet meer bruikbaar doordat de source-code van de website volledig is veranderd. De code die gebruikt is om de data te verkrijgen is in de markdown cell gezet. Na de uitleg van de code lezen we het csv bestand in dat gemaakt is na het uitvoeren van de code.

```py
# Aanmaken van lijsten om de data in te stoppen
titel = []
jaartal = []
lengte = []
imdb_scores = []
meta_scores = []
stemmen = []
us_omzet = []
omschrijving = []
certificaat = []
genre = []
regisseur = []
sterren = []

# Verkrijgen van de engelse namen van films
en_titel = {'Accept-Language': 'en-US, en;q=0.5'}

# Aanmaken van lijst voor pagina's
pagina = np.arange(1, 1001, 50)

# Aanmaken van de URL
url = 'https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&start='

# Zorgen dat de scraping voor elke 50 gaat
for p in pagina:
    # Pakken van URL
    p = requests.get(
        url + str(p) + '&ref_=adv_nxt', headers=en_titel
        )

    # Beginnen van de soup
    soup = BeautifulSoup(p.text, 'html.parser')

    # Zoeken van alle films op de pagina
    films = soup.find_all('div', class_='lister-item mode-advanced')

    # Wachtijd van 2 tot 10 seconden
    sleep(randint(2, 10))

    for item in films:
        # Titel
        titel.append(item.h3.a.text)

        # Jaartal
        jaartal.append(item.h3.find('span', class_='lister-item-year').text)

        # Regisseur
        regisseur.append(item.find('p', class_='').find('a').text)

        # Hoofd-acteurs
        acteurs = item.find('p', class_='').find_all('a')
        stars = []
        for tag in acteurs[-4:]:
            stars.append(tag.text)
        sterren.append(stars)

        # Leeftijd certificatie
        cert = (item.find('span', class_='certificate').text
                if item.p.find('span', class_='certificate') else 'NotFound')
        certificaat.append(cert)

        # Lengte
        runtime = (item.find('span', class_='runtime').text
                    if item.p.find('span', class_='runtime') else 'NotFound')
        lengte.append(runtime)

        # Genre
        gen = (item.find('span', class_='genre').text
                if item.p.find('span', class_='genre') else 'NotFound')
        genre.append(gen)

        # IMDb rating
        imdb_scores.append(float(item.strong.text))

        # meta_scores
        m_score = (item.find('span', class_='metascore').text
                    if item.find('span', class_='metascore') else 'NotFound')
        meta_scores.append(m_score)

        # Omschrijving
        desc = item.find_all('p', class_='text-muted')
        omschrijving.append(desc[1].text)

        # Stemmen en Omzet
        so = item.find_all('span', attrs={'name':'nv'})
        stemmen.append(so[0].text)
        us_omzet.append(so[1].text if len(so) > 1 else '-')

    print("-- Iteratie van loop voltooid --")

# Aanmaken van een dataframe
films = pd.DataFrame(
    {'Film' : titel,
     'Omschrijving' : omschrijving,
     'Regisseur' : regisseur,
     'Hoofd Acteurs' : sterren,
     'Age_Rating' : certificaat,
     'Genres' : genre,
     'Jaar' : jaartal,
     'Minuten' : lengte,
     'IMDb_Score' : imdb_scores,
     'Meta_Score' : meta_scores,
     'Stemmen' : stemmen,
     'Omzet (in M)' : us_omzet}
)

# Data preprocessing van de films dataframe
# Opschonen van de omschrijving kolom
films['Omschrijving'] = films['Omschrijving'].str.strip()

# Opschonen van de acteurs kolom
films['Hoofd Acteurs'] = films['Hoofd Acteurs'].astype(str)\
                            .replace({'\'': '', '\[|\]': ''}, regex=True)

# Opschonen van de genres kolom
films['Genres'] = films['Genres'].str.strip()

# Opschonen van de Jaar kolom
films['Jaar'] = films['Jaar'].str.extract('(\d+)').astype(int)

# Opschonen van de Minuten kolom
films['Minuten'] = films['Minuten'].str.extract('(\d+)').astype(int)

# Opschonen van de Meta_score kolom
films['Meta_Score'] = films['Meta_Score'].str.extract('(\d+)')

# Omzetten naar float en NotFound veranderen naar NaN
films['Meta_Score'] = pd.to_numeric(films['Meta_Score'], errors='coerce')

# Opschonen van de Stemmen kolom
films['Stemmen'] = films['Stemmen'].str.replace(',', '').astype(int)

# Opschonen van Omzet kolom
# Weghalen van '$' en 'M'
films['Omzet (in M)'] = films['Omzet (in M)'].map(lambda x: x.lstrip('$').rstrip('M'))

# Omzetten naar float en NotFound veranderen naar NaN
films['Omzet (in M)'] = pd.to_numeric(films['Omzet (in M)'], errors='coerce')

films.to_csv('IMDb_data.csv')
```

Met de bovenstaande code is het gelukt om de data van de voorgaande versie van IMDb te extraheren en de data alvast te preprocessen. Deze data is vervolgens in een CSV bestand gezet die nu zal worden ingeladen.

In [3]:
# Inladen van IMDb Data
films = pd.read_csv('IMDb_data.csv', index_col=0)

# Tonen data
display(films.dtypes)
display(films)

Film              object
Omschrijving      object
Regisseur         object
Hoofd Acteurs     object
Age_Rating        object
Genres            object
Jaar               int64
Minuten            int64
IMDb_Score       float64
Meta_Score       float64
Stemmen            int64
Omzet (in M)     float64
dtype: object

,Film,Omschrijving,Regisseur,Hoofd Acteurs,Age_Rating,Genres,Jaar,Minuten,IMDb_Score,Meta_Score,Stemmen,Omzet (in M)
0,The Shawshank Redemption,"Over the course of several years, two convicts...",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",R,Drama,1994,142,9.3,82.0,2818589,28.34
1,The Godfather,"Don Vito Corleone, head of a mafia family, dec...",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Diane Ke...",R,"Crime, Drama",1972,175,9.2,100.0,1964393,134.97
2,The Dark Knight,When the menace known as the Joker wreaks havo...,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",PG-13,"Action, Crime, Drama",2008,152,9.0,84.0,2800102,534.86
3,Schindler's List,"In German-occupied Poland during World War II,...",Steven Spielberg,"Liam Neeson, Ralph Fiennes, Ben Kingsley, Caro...",R,"Biography, Drama, History",1993,195,9.0,95.0,1416792,96.90
4,The Lord of the Rings: The Return of the King,Gandalf and Aragorn lead the World of Men agai...,Peter Jackson,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",PG-13,"Action, Adventure, Drama",2003,201,9.0,94.0,1930207,377.85
...,...,...,...,...,...,...,...,...,...,...,...,...
995,When Marnie Was There,"Due to 12-year-old Anna's asthma, she's sent t...",Hiromasa Yonebayashi,"Sara Takatsuki, Kasumi Arimura, Nanako Matsush...",PG,"Animation, Drama, Family",2014,103,7.6,72.0,45579,0.77
996,Control,"A profile of Ian Curtis, the enigmatic singer ...",Anton Corbijn,"Sam Riley, Samantha Morton, Craig Parkinson, A...",R,"Biography, Drama, Music",2007,122,7.6,78.0,67786,0.87
997,Philomena,A world-weary political journalist picks up th...,Stephen Frears,"Judi Dench, Steve Coogan, Sophie Kennedy Clark...",PG-13,"Biography, Comedy, Drama",2013,98,7.6,77.0,102917,37.71
998,Shine,"Pianist David Helfgott, driven by his father a...",Scott Hicks,"Geoffrey Rush, Armin Mueller-Stahl, Justin Bra...",PG-13,"Biography, Drama, Music",1996,105,7.6,87.0,55997,35.81


Met deze dataframe is het mogelijk om stappen voor het feature engineering te testen, voordat deze in de pipeline komen.

[Terug naar Inhoudsopgave](#begin)

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:40px; font-weight:bold;">Database: Movie Summaries</div>
    <a name='db'></a>
</div>

Voor het Database onderdeel van de opgave, is er gebruik gemaakt van een online downloadbare bron genaamd de Movie Summary Corpus. Deze databron bevat verschillende informatie van ongeveer 42000 films. Er staat informatie over de verschillende karakters in de films, een uitgebreide omschrijving van het plot en metadata over de film. Voor dit onderzoek zal de data over de karakters niet nodig zijn en dus zal deze niet worden ingeladen. De onderstaande stukken uitleg zijn voor de data die wij gaan gebruiken. Met dank aan de README.txt die bij de download van de data zat, is het gemakkelijker om de data en de structuur te begrijpen. Voor het inladen van elk bestand is de README gebruikt om de kolomnamen aan te maken en de data op de juiste manier in te laden.
<br>
<br>
<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:20px; font-weight:bold;">Bestand: plot_summaries.txt</div>
    <a name='db'></a>
</div>

Door middel van pd.read_csv is het mogelijk om text bestanden in te lezen. Bij het bekijken van het tekst bestand werd duidelijk dat de scheiding van kolommen is aangegeven door een tab. Verder is de README informatie gebruikt om de kolomnamen te maken:
- Plot summaries of 42,306 movies extracted from the November 2, 2012 dump of English-language Wikipedia.  Each line contains the Wikipedia movie ID (which indexes into movie.metadata.tsv) followed by the summary. (Gekopieerd van README.txt)
<br>
<br>
<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:20px; font-weight:bold;">Bestand: movie.metadata.tsv</div>
</div>

Ook bij dit tsv bestand kan er gebruik worden gemaakt van de pd.read_csv functie. Bij het lezen van de README is duidelijk neergezet hoe de structuur van dit bestand eruit ziet.

Metadata for 81,741 movies, extracted from the Noverber 4, 2012 dump of Freebase.  Tab-separated; columns: (Gekopieerd van README.txt)

1. Wikipedia movie ID
2. Freebase movie ID
3. Movie name
4. Movie release date
5. Movie box office revenue
6. Movie runtime
7. Movie languages (Freebase ID:name tuples)
8. Movie countries (Freebase ID:name tuples)
9. Movie genres (Freebase ID:name tuples)

<br>
<br>

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:20px; font-weight:bold;">Data inladen naar database</div>
</div>

Om de data in te kunnen lezen, wordt er gebruik gemaakt van een database. Deze database zal worden aangemaakt door de corpus van het internet te downloaden, om deze vervolgens in een database in te laden. Deze taken worden gedaan met behulp van verschillende libaries, met name urllib en sqlite3. Om al deze stappen uit te voeren worden er eerst een aantal functies aangemaakt. Elke functie heeft zijn eigen doel en samen geven ze een gestreamlinede weg naar het maken van de database.

In [9]:
def download_and_extract(url, folder):
    """
    Deze functie download een gecomprimeerd archief van de
    opgegeven URL en extraheerd de data naar een opgegeven map.

    Parameters:
    ----------
    archive_url : str
        De URL van het te downloaden archief.

    target_folder : str
        Het pad naar de doelmap waarin de
        bestanden worden geëxtraheerd.

    Returns:
    ----------
    None
    """
    # Openen van de URL en lezen van de response
    response = urllib.request.urlopen(url)
    tar_data = BytesIO(response.read())

    # Downloaden en extraheren van de bestanden
    with tarfile.open(fileobj=tar_data, mode='r:gz') as tar:
        tar.extractall(path=folder)

def transfer_dataframe_to_db(df, table_name, db_path):
    """
    Overzetten van een Pandas DataFrame naar een
    SQLite-database tabel.

    Parameters:
    ----------
    df : pandas.DataFrame
        Het DataFrame dat moet worden overgezet.

    table_name : str
        De naam van de tabel in de database.

    db_path : str
        Het pad naar de SQLite-database.

    Returns:
    ----------
    None
    """
    # Verbinden met de database
    conn = sqlite3.connect(db_path)

    # Aanmaken van de tabel
    df.to_sql(name=table_name, con=conn, if_exists='replace', index=False)

    # Pushen van verandering naar Database en stoppen connectie
    conn.commit()
    conn.close()

def query_exe(db_path, query):
    """
    Uitvoeren van een SQL-query op de opgegeven SQLite-database
    en de resultaten omzetten naar een Pandas DataFrame.

    Parameters:
    db_path : str
        Het pad naar de SQLite-database.

    query : str
        De SQL-query die moet worden uitgevoerd.

    Returns:
    ----------
    df : pandas.DataFrame
        Het resultaat van de query als een DataFrame.
    """
    # Verbinden met de database
    conn = sqlite3.connect(db_path)

    # Query inlezen tot dataframe
    df = pd.read_sql_query(query, conn)

    # Sluiten van de connectie met database
    conn.close()

    return df

Met de functies aangemaakt kunnen we de nodige parameters definieeren om de data te downloaden en in te laden in de database. 

In [7]:
# URL download
url = "http://www.cs.cmu.edu/~ark/personas/data/MovieSummaries.tar.gz"

# Folder path
folder = "MovieSummaries"

# Database naam
database = 'movie_database.db'

# Kolommen van de data
kolommen = {
    "movie.metadata": ['Wikipedia_ID', 'Freebase_ID', 'Film',
                       'Release_date', 'Box_office', 'Lengte',
                       'Talen', 'Landen', 'Genres'],
    "character.metadata": ['Wikipedia_ID', 'Freebase_ID', 'Release_date',
                           'Char_Name', 'Actor_DoB', 'Actor_gender', 'Actor_h',
                           'Actor_eth', 'Actor_Name', 'Actor_age',
                           'Freebase_map_ID', 'Freebase_ch_ID', 'Freebase_ac_ID'],
    "plot_summaries": ['Wikipedia_ID', 'Summary'],
    "name.clusters": ['Name', 'Freebase_ID'],
    "tvtropes.clusters": ['Trope', 'Character_Data']
}

Met gebruik van deze vier parameters is het mogelijk om de nodige functies voor het downloaden van de data en het inladen in de database uit te gaan voeren.

In [8]:
# Uitvoeren van de download en extract stap
download_and_extract(url, folder)

# Loopen over de folder met geextraheerde data
for root, dirs, files in os.walk(folder):
    for file in files:
        # Aanmaken van bestand locatie, type en de tabel naam
        file_path = os.path.join(root, file)
        file_type = 'tsv' if file.endswith(".tsv") else 'txt'
        table_name = os.path.splitext(file)[0]

        # Excluderen van README.txt
        if file != "README.txt":
            print(f"Processen van {file_type.upper()} bestand: {file_path}")

            # Aangeven welke kolommen worden gebruikt
            custom_kols = kolommen.get(table_name, None)

            # Aanmaken van dataframe
            df = pd.read_csv(file_path, sep='\t', names=custom_kols)

            # Dataframe transporteren naar database
            transfer_dataframe_to_db(df, table_name, database)
            print(f"Tabel '{table_name}' is toegevoegd")

Processing TSV file: MovieSummaries\MovieSummaries\character.metadata.tsv
Table 'character.metadata' is added
Processing TSV file: MovieSummaries\MovieSummaries\movie.metadata.tsv
Table 'movie.metadata' is added
Processing TXT file: MovieSummaries\MovieSummaries\name.clusters.txt
Table 'name.clusters' is added
Processing TXT file: MovieSummaries\MovieSummaries\plot_summaries.txt
Table 'plot_summaries' is added
Processing TXT file: MovieSummaries\MovieSummaries\tvtropes.clusters.txt
Table 'tvtropes.clusters' is added


Nu alle data als tabel is toegevoegd in de database, worden er een paar queries uitgevoerd. Op deze manier is het aan te tonen dat de code heeft gewerkt.

In [3]:
# Aanmaken van query
query = """
    SELECT Wikipedia_ID,
           Film,
           Lengte,
           Genres
        FROM 'movie.metadata'
"""

# Aanmaken van dataframe door query in te lezen
df = query_exe(database, query)

# Tonen van dataframe
display(df)

,Wikipedia_ID,Titel,Lengte,Genres
0,975900,Ghosts of Mars,98.0,"{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,Getting Away with Murder: The JonBenét Ramsey ...,95.0,"{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,Brun bitter,83.0,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,White Of The Eye,110.0,"{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,A Woman in Flames,106.0,"{""/m/07s9rl0"": ""Drama""}"
...,...,...,...,...
81736,35228177,Mermaids: The Body Found,120.0,"{""/m/07s9rl0"": ""Drama""}"
81737,34980460,Knuckle,96.0,"{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0..."
81738,9971909,Another Nice Mess,66.0,"{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}"
81739,913762,The Super Dimension Fortress Macross II: Lover...,150.0,"{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ..."


In [4]:
# Aanmaken van query
query = """
    SELECT *
        FROM 'plot_summaries'
"""

# Aanmaken van dataframe door query in te lezen
df_sum = query_exe(database, query)

# Tonen van dataframe
display(df_sum)

,Wikipedia_ID,Omschrijving
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...
...,...,...
42298,34808485,"The story is about Reema , a young Muslim scho..."
42299,1096473,"In 1928 Hollywood, director Leo Andreyev look..."
42300,35102018,American Luthier focuses on Randy Parsons’ tra...
42301,8628195,"Abdur Rehman Khan , a middle-aged dry fruit se..."


Met deze dataframes is het mogelijk om stappen voor het preprocessen en feature engineering te testen, voordat deze in de pipeline komen.

[Terug naar Inhoudsopgave](#begin)

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:40px; font-weight:bold;">Preprocessing en Feature Engineering</div>
    <a name='tr'></a>
</div>

Om te kunnen werken met de ingeladen data is het noodzakelijk dat er toepassingen op worden uitgevoerd. De eerste vorm van toepassingen heet preprocessing. Deze stap is verantwoordelijk voor het volledig opschonen van de data. De tweede vorm heet Feature engineering. Deze vorm van toepassingen slaan op het bruikbaar maken van de data voor Machine Learning. Hierbij moeten er vaak nieuwe kolommen worden aangemaakt, of moeten oude kolommen worden herschreven. In deze sectie van het notebook gaan we kijken naar welke transformaties de data nodig zal hebben om bruikbaar te zijn voor Machine Learning.

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:20px; font-weight:bold;">Preprocessing</div>
</div>

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:20px; font-weight:bold;">Feature Engineering</div>
</div>

Nu alle data is gepreprocessed, kan er worden gewerkt aan het maken van de features. Een groot onderdeel bij Feature Engineering is het toepassen van de juiste technieken op de juiste typen data. In onze datasets staat data in de vorm van woorden en teksten, deze zullen moeten worden aangepast om bruikbaar te worden gemaakt in ML.

In [5]:
def process_text_columns(df, kolommen):
    """
    Deze functie processed de opgegeven kolommen, zodat de
    tekst bruikbaarder is. De opgegeven kolommen moeten
    hiervoor wel tekstuele data bevatten.

    Parameters:
    ----------
    df : pandas.DataFrame
        Het dataframe waarop de aanpassingen worden uitgevoerd.

    kolommen : list or str
        De kolom(men) waarop de aanpassingen worden uitgevoerd.

    Returns:
    ----------
    df_nlp : pandas.DataFrame
        Het dataframe waarop de toepassing zijn uitgevoert.
    """
    # Het selecteren van engelse stopwoorden voor in de tekst
    stopwoorden = set(stopwords.words('english'))

    # Aanmaken van process text functie om apply te kunnen gebruiken
    def process_text(text):
        """
        Deze functies past verschillende taken
        toe op basis van NLP technieken

        Parameters:
        ----------
        text : str
            Een str aan tekst in een dataframe kolom

        Returns:
        ----------
        processed_text : str
            Een str met de aangepaste tekst
        """
        # Aanmaken van tokens in de tekst
        tokens = re.findall(r'\w+', text)

        # Het weghalen van punctuatie binnen de tekst
        no_punctuations = ' '.join(re.sub(r'\W', ' ', token) for token in tokens)

        # Het veranderen van alle tekst naar kleine letters
        lower_text = no_punctuations.lower()

        # Het verwijderen van de stopwoorden
        processed_text = ' '.join(word for word in lower_text.split() if word not in stopwoorden)

        return processed_text

    # Loopen over de kolommen van het opgegeven dataframe
    for kol in kolommen:
        df[kol] = df[kol].apply(process_text)

    return df

,Wikipedia_ID,Omschrijving
0,23890098,shlykov hard working taxi driver lyosha saxoph...
1,31186339,nation panem consists wealthy capitol twelve p...
2,20663735,poovalli induchoodan sentenced six years priso...
3,2231378,lemon drop kid new york city swindler illegall...
4,595909,seventh day adventist church pastor michael ch...
...,...,...
42298,34808485,story reema young muslim schoolgirl malabar lo...
42299,1096473,1928 hollywood director leo andreyev looks pho...
42300,35102018,american luthier focuses randy parsons transfo...
42301,8628195,abdur rehman khan middle aged dry fruit seller...


Nu de functie voor de NLP technieken is aangemaakt kan deze worden ingezet om de tekst kolommen aan te passen.

In [ ]:
# Invoeren van de kolommen die aangepast worden
kolommen = ['Summary']

# Uitvoeren van de aanpassingen
processed_df = process_text_columns(df_sum, kolommen)

# Tonen van het aangepaste dataframe
display(processed_df)

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:40px; font-weight:bold;">Opzetten van de Pipeline</div>
    <a name='pipe'></a>
</div>

Om te zorgen dat alle data op de juiste manier word ingeladen en getransformeerd maken we gebruik van een pipeline. De pipeline is eigenlijk een class vol met functies die ons helpt om data gemakkelijk in te laden en te transformeren. De functies in de pipeline zijn zo robuust mogelijk opgesteld, zodat er geen problemen zijn bij het inladen en verwerken van een andere databron.

In [2]:
class ETL_Pipeline:
    """
    Class voor een Extract-Transform-Load Pipeline.
    Deze class kan nodige data van een database, API of
    webscraping bron halen. Deze data kan vervolgens
    getransformeerd worden, waarna de data kan worden
    ingeladen in een pandas DataFrame
    """
    def __init__(self, db_source=None, query=None, api_source=None, csv_file=None):
        """
        Initiator van de class. De initiator neemt bepaalde
        waarden op. Met behulp van deze waarden kunnen de
        functies worden uitgevoerd.
        """
        self.db_source = db_source
        self.query = query
        self.api_source = api_source
        self.csv_file = csv_file
        self.data_frame = pd.DataFrame()

    def extract_db(self):
        """
        Deze functie haalt data op uit een lokale database.
        """
        # Verbinden met de database
        conn = sqlite3.connect(self.db_source)

        # Uitvoeren van de query op de database
        db_data = pd.read_sql_query(self.query, conn)

        # Sluiten van de verbinding met de database
        conn.close()

        return db_data

    def extract_api(self):
        """
        Deze functie haalt data op uit ruwe bronnen.
        """
        api_data = pd.DataFrame(...)

        return api_data

    def extract_csv(self):
        """
        Deze functie haalt data op uit ruwe bronnen.
        """
        # Inlezen van csv data
        csv_data = pd.read_csv(self.csv_file, index_col=0)

        return csv_data

    def transform_data(self, data_frames):
        """
        Deze functie transformeerd de data op basis van
        de gegeven methoden.
        """
        transformed_data = pd.DataFrame(...)

        return transformed_data

    def load_data(self):
        """
        Deze functie laad de data in naar een
        pandas.DataFrame.
        """
        # Maken van lege data indien databron niet gebruikt
        db_data = None
        api_data = None
        csv_data = None

        # Extraheer data van gebruikte bronnen
        if self.db_source:
            db_data = self.extract_db()
        if self.api_source:
            api_data = self.extract_api()
        if self.csv_file:
            csv_data = self.extract_csv()

        # Zet alle databronnen in een lijst
        all_data_frames = [db_data, api_data, csv_data]

        # Transformeer en merge alle data
        transformed_data = self.transform_data(all_data_frames)

        # Toewijzen van de data aan de data_frame class attribuut
        self.data_frame = transformed_data

        return transformed_data

Nu de Pipeline is aangemaakt, kunnen de parameters worden gedefined. De parameters zijn opzettelijk los neergezet, om de Pipeline niet vast te zetten in het geval van kleine aanpassingen aan de locaties van bestanden.

In [ ]:
# Invoeren van de DataBase naam
db_source = "movie_databse.db"

# Invoeren van query op DataBase
query = """
    SELECT mm.Wikipedia_ID,
           mm.Film,
           ps.Summary,
           mm.Lengte,
           mm.Genres
    FROM 'movie.metadata' AS mm
        
    JOIN 'plot_summaries' AS ps
        ON mm.Wikipedia_ID = ps.Wikipedia_ID
"""

# Invoeren van de API link
api_source = "API___URL"

# Invoeren van csv file path
csv_file = "IMDb_data.csv"

Met alle nodige parameters gedefineerd, kan de pipeline worden gebruikt om de data in te laden.

In [ ]:
etl = ETL_Pipeline(db_source, query, api_source, csv_file)
df = etl.load_data()

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:40px; font-weight:bold;">Aantonen dat de Pipeline werkt</div>
    <a name='toon'></a>
</div>

<div style="background-color:#600170; color:#fff; padding:10px; border-radius:5px; display: flex; justify-content: center; align-items: center; position: relative;">
    <div style="font-size:40px; font-weight:bold;">Bronnen</div>
    <a name='bron'></a>
</div>

Please cite this paper if you write any papers involving the use of the data (van de Movie Summary Corpus):

    Learning Latent Personas of Film Characters
    David Bamman, Brendan O'Connor, and Noah A. Smith
    ACL 2013, Sofia, Bulgaria, August 2013

